# Overview
- targetの情報を用いてgroupを作成する
- nb003では、696個にグルーピングした。
- all_target = 0 のグループがかなり多いのでそれを細かく分割しようと思う。
- how to
    - group_0とgroup_not0分ける
    - gourp_0を5fold, group_not0をgroup5foldする
    - fold情報を合体させる

# Const

In [1]:
NB = '004'
PATH_TRAIN = '../data_ignore/input/train_features.csv'
PATH_TRAIN_SCORED = '../data_ignore/input/train_targets_scored.csv'
PATH_TRAIN_NONSCORED = '../data_ignore/input/train_targets_nonscored.csv'
PATH_SUB = '../data_ignore/input/sample_submission.csv'
PATH_TEST = '../data_ignore/input/test_features.csv'
PATH_GROUP = '../data_ignore/output_nb/nb003/group.csv'

# Import everything I need :)

In [50]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fastprogress import progress_bar
from sklearn.model_selection import GroupKFold, KFold

# My function

# Preparation

set

In [51]:
sns.set()
pd.set_option('display.max_columns', 1000)
warnings.filterwarnings('ignore')

<br>

load datasets

In [4]:
# train_features = pd.read_csv(PATH_TRAIN)
# train_scored = pd.read_csv(PATH_TRAIN_SCORED)
# train_nonscored = pd.read_csv(PATH_TRAIN_NONSCORED)
# sub = pd.read_csv(PATH_SUB)
# test_features = pd.read_csv(PATH_TEST)
group = pd.read_csv(PATH_GROUP)

# Grouping

## all_target = 0 のグループNoを0とする

In [5]:
mask_0 = group['group'] == 0
mask_1 = group['group'] == 1
group['group'][mask_0] = 1
group['group'][mask_1] = 0
group['group'].value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


0      5530
13      718
25      363
53      337
47      328
       ... 
695       1
649       1
671       1
519       1
692       1
Name: group, Length: 696, dtype: int64

---> 0が一番多い(やりたいことできた)

<br>

save gourp.csv

In [6]:
filename = '../data_ignore/output_nb/nb004/group.csv'
file_dir = os.path.dirname(filename)
if not os.path.exists(file_dir):
    os.makedirs(file_dir)
group.to_csv(filename, index=False)

# gorup = 0 を分解しつつ、5foldに分ける
- やりたいこと
    - group_0とgroup_not0分ける
    - gourp_0を5fold, group_not0をgroup5foldする
    - fold情報を合体させる

<br>

group = 0 の分解

In [13]:
mask_0 = group['group'] == 0
group_0 = group[mask_0]
splitter = KFold(n_splits=5, random_state=0, shuffle=True)

In [8]:
df_fold_0 = pd.DataFrame()
df_fold_0['fold'] = np.zeros(len(group_0)).astype(int)
for i_fold, (idx_trn, idx_val)  in enumerate(splitter.split(group_0)):
    df_fold_0['fold'][idx_val] = int(i_fold + 1)

In [9]:
df_fold_0

,fold
0,5
1,2
2,3
3,3
4,1
...,...
5525,1
5526,1
5527,5
5528,3


<br>

group_not0の分解

In [14]:
mask_not0 = group['group'] != 0
group_not0 = group[mask_not0]
splitter = GroupKFold(n_splits=5)

In [11]:
df_fold_not0 = pd.DataFrame()
df_fold_not0['fold'] = np.zeros(len(group_not0)).astype(int)
for i_fold, (idx_trn, idx_val)  in enumerate(splitter.split(group_not0, groups=group_not0['group'].values)):
#     display(group_not0.iloc[idx_val, :])
#     print(group_not0.iloc[idx_val, :]['group'].nunique())
    df_fold_not0['fold'][idx_val] = int(i_fold + 1)

In [12]:
df_fold_not0

,fold
0,2
1,2
2,1
3,2
4,2
...,...
18279,3
18280,2
18281,5
18282,2


<br>
結合

In [53]:
df_fold = pd.DataFrame()
df_fold['sig_id'] = group['sig_id'].values
df_fold['fold'] = np.zeros(len(group)).astype(int)
df_fold['fold'][mask_0] = df_fold_0['fold'].values
df_fold['fold'][mask_not0] = df_fold_not0['fold'].values

In [26]:
df_fold

,sig_id,fold
0,id_000644bb2,2
1,id_000779bfc,5
2,id_000a6266a,2
3,id_0015fd391,2
4,id_001626bd3,1
...,...,...
23809,id_fffb1ceed,5
23810,id_fffb70c0c,2
23811,id_fffc1c3f4,3
23812,id_fffcb9e7c,5


# Summary

In [48]:
def get_696_strategy_fold(group, n_splits):
    # group == 0 (all_target=0)の分解
    mask_0 = group['group'] == 0
    group_0 = group[mask_0]
    splitter = KFold(n_splits=n_splits, random_state=0, shuffle=True)
    df_fold_0 = pd.DataFrame()
    df_fold_0['fold'] = np.zeros(len(group_0)).astype(int)
    for i_fold, (idx_trn, idx_val)  in enumerate(splitter.split(group_0)):
        df_fold_0['fold'][idx_val] = int(i_fold + 1)

    # group != 0 の分解
    mask_not0 = group['group'] != 0
    group_not0 = group[mask_not0]
    splitter = GroupKFold(n_splits=n_splits)
    df_fold_not0 = pd.DataFrame()
    df_fold_not0['fold'] = np.zeros(len(group_not0)).astype(int)
    for i_fold, (idx_trn, idx_val)  in enumerate(splitter.split(group_not0, groups=group_not0['group'].values)):
        df_fold_not0['fold'][idx_val] = int(i_fold + 1)

    # fold情報の結合
    df_fold = pd.DataFrame()
    df_fold['sig_id'] = group['sig_id'].values
    df_fold['fold'] = np.zeros(len(group)).astype(int)
    df_fold['fold'][mask_0] = df_fold_0['fold'].values
    df_fold['fold'][mask_not0] = df_fold_not0['fold'].values
    return df_fold

In [78]:
df = get_696_strategy_fold(group, 5)
df['fold'].value_counts()

4    4763
3    4763
2    4763
1    4763
5    4762
Name: fold, dtype: int64